In [1]:
import numpy as np
import xarray as xr
import xrft
import matplotlib.pyplot as plt
import matplotlib.colors as pltcolors
import cmocean as cm
 
def_colours = [(0.00000, 0.44706, 0.69804),
          (0.00000, 0.61961, 0.45098),
          (0.94118, 0.89412, 0.25882),
          (0.90196, 0.62353, 0.00000),
          (0.83529, 0.36863, 0.00000),
          (0.80000, 0.47451, 0.65490),
          (0.33725, 0.70588, 0.91373)]

In [2]:
path = '/scratch/mp6191/GeophysicalFlows_expts/MovieRuns'
expts = ['/kappa02_notopo', '/kappa02_kt25_h4', '/kappa02_kt25_h6', '/kappa02_kt25_h12', '/kappa02_GJ_h4', '/kappa02_GJ_h8']
names = [expt[1:] for expt in expts]
paths = [path + expt + '/output' + expt + '.nc' for expt in expts]

# Import simulations
expt_dict = dict()
for i in range(len(expts)):
    name = names[i]
    path = paths[i]
    ds = xr.open_dataset(path)
    expt_dict[name] = ds

In [3]:
nx = 1024
ny = nx
nk = nx
nl = ny
nz = 2
Ld = 15e3
ld = 2 * np.pi * Ld
L = 25 * ld
f = 1e-4
beta = 0.
delta = 1

F1 = Ld ** -2 / (1. + delta)
F2 = delta * F1

dk = 2 * np.pi / L
k = dk * np.append( np.arange(0, nx / 2), np.arange(- nx / 2, 0) )
k = np.fft.fftshift(k)
l = k
kk, ll = np.meshgrid(k, l)
K2 = kk ** 2 + ll ** 2

# Stretching matrix
S = np.array([[-F1, F1],
              [F2, -F2]])

a = np.ma.zeros((nz, nz, nl, nk), np.dtype('float64'))
det_inv = np.ma.masked_equal(K2 * (K2 + F1 + F2), 0.) ** -1
a[0,0] = -(K2 + F2) * det_inv
a[0,1] = -F1 * det_inv
a[1,0] = -F2 * det_inv
a[1,1] = -(K2 + F1) * det_inv
a = a.filled(1e20)


def psih_func(a, q):
    '''
    Function for calculating Fourier space stream function from PV field at all times.
    '''
    
    qh = xrft.fft(q, dim = ['x', 'y'], true_phase = False, true_amplitude = False)

    a = xr.DataArray(data = a,
                         dims = ['i', 'lev', 'freq_y', 'freq_x'],
                         coords = dict(
                         freq_x = ('freq_x', qh.freq_x.values),
                         freq_y = ('freq_y', qh.freq_y.values),
                         lev = ('lev', [1, 2]),
                         i = ('i', [1, 2]))
                    )

    psih = xr.dot(a, qh, dims = ['lev']).rename({'i' : 'lev'})
    
    return psih

def psi_func(psih, q):
    '''
    Function for calculating real space stream function from Fourier space stream function at all times.
    '''
        
    psi = xrft.ifft(psih, dim = ['freq_x', 'freq_y'], true_phase = False, true_amplitude = False).real.rename('psi')
    psi = psi.assign_coords(dict(x = ('x', q.x.values), y = ('y', q.y.values)))
    
    return psi

def zeta_func(K2, psih, q):
    '''
    Function for calculating real space vorticity from Fourier space stream function at all times.
    '''
    
    zetah = - K2[np.newaxis, :, :, np.newaxis] * psih
    zeta = xrft.ifft(zetah, dim = ['freq_x', 'freq_y'], true_phase = False, true_amplitude = False).real.rename('zeta')
    zeta = zeta.assign_coords(dict(x = ('x', q.x.values), y = ('y', q.y.values)))
    
    return zeta

def u_func(ll, psih, q):
    '''
    Function for calculating zonal velocity in real space from Fourier space stream funciton function at all times.
    '''
    
    uh = -1j * ll[np.newaxis, :, :, np.newaxis] * psih
    u = xrft.ifft(uh, dim = ['freq_x', 'freq_y'], true_phase = False, true_amplitude = False).real.rename('u')
    u = u.assign_coords(dict(x = ('x', q.x.values), y = ('y', q.y.values)))
    
    return u

def v_func(kk, psih, q):
    '''
    Function for calculating meridional velocity in real space from Fourier space stream funciton function at all times.
    '''
    
    vh = 1j * kk[np.newaxis, :, :, np.newaxis] * psih
    v = xrft.ifft(vh, dim = ['freq_x', 'freq_y'], true_phase = False, true_amplitude = False).real.rename('v')
    v = v.assign_coords(dict(x = ('x', q.x.values), y = ('y', q.y.values)))
    
    return v

In [23]:
from xmovie import Movie
import matplotlib.ticker as tkr

def numfmt(x, pos):
    s = '{}'.format(int(x))
    return s
fmt = tkr.FuncFormatter(numfmt)

# This defines a custom plotting function that will plot each variable next to each other

Ld = 15e3
U0 = 0.01

fontsize = 16
def custom_plotfunc(ds, fig, i, *args, **kwargs):
    (ax1, ax2) = fig.subplots(ncols = 2, sharey=True)
    fig.set_figheight(7)
    fig.set_figwidth(18)
    
    t = ds.time.values
    ti = format(t[i] * U0 / Ld, '.2f')
    x = ds.x.values / Ld
    y = ds.y.values / Ld
    xx, yy = np.meshgrid(x, y)
    zeta1 = zeta.isel(time = i).isel(lev = 0)
    zeta2 = zeta.isel(time = i).isel(lev = 1)
    
    vmax = 300
    vmin = -vmax

    cmap = cm.cm.curl
    
    ax = ax1
    im = ax.pcolormesh(xx, yy, zeta1, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, upper layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel(r'$y / \lambda$', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

    vmax = 150
    vmin = -vmax
    
    ax = ax2
    im = ax.pcolormesh(xx, yy, zeta2, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, lower layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel('', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

/ext3/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
zeta = zeta.rename({'t': 'time'})
mov_custom = Movie(zeta, custom_plotfunc, input_check = False)
mov_custom.save(name + '_zeta_layer.mp4', framerate = 10, overwrite_existing = True)
zeta = zeta.rename({'time': 't'})

/ext3/miniconda3/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(


Movie created at kappa02_notopo_zeta_layer.mp4


In [25]:
name = 'kappa02_kt25_h4'
ds = expt_dict[name]
q = ds.q
psih = psih_func(a, q)
zeta = zeta_func(K2, psih, q) * Ld / U0

/ext3/miniconda3/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


In [26]:
from xmovie import Movie
import matplotlib.ticker as tkr

def numfmt(x, pos):
    s = '{}'.format(int(x))
    return s
fmt = tkr.FuncFormatter(numfmt)

# This defines a custom plotting function that will plot each variable next to each other

Ld = 15e3
U0 = 0.01

fontsize = 16
def custom_plotfunc(ds, fig, i, *args, **kwargs):
    (ax1, ax2) = fig.subplots(ncols = 2, sharey=True)
    fig.set_figheight(7)
    fig.set_figwidth(18)
    
    t = ds.time.values
    ti = format(t[i] * U0 / Ld, '.2f')
    x = ds.x.values / Ld
    y = ds.y.values / Ld
    xx, yy = np.meshgrid(x, y)
    zeta1 = zeta.isel(time = i).isel(lev = 0)
    zeta2 = zeta.isel(time = i).isel(lev = 1)
    
    vmax = 300
    vmin = -vmax

    cmap = cm.cm.curl
    
    ax = ax1
    im = ax.pcolormesh(xx, yy, zeta1, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, upper layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel(r'$y / \lambda$', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

    vmax = 150
    vmin = -vmax
    
    ax = ax2
    im = ax.pcolormesh(xx, yy, zeta2, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, lower layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel('', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

In [27]:
zeta = zeta.rename({'t': 'time'})
mov_custom = Movie(zeta, custom_plotfunc, input_check = False)
mov_custom.save(name + '_zeta_layer.mp4', framerate = 10, overwrite_existing = True)
zeta = zeta.rename({'time': 't'})

/ext3/miniconda3/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(


Movie created at kappa02_kt25_h4_zeta_layer.mp4


In [4]:
Ld = 15e3
U0 = 1e-2

In [5]:
name = 'kappa02_kt25_h6'
ds = expt_dict[name]
q = ds.q
psih = psih_func(a, q)
zeta = zeta_func(K2, psih, q) * Ld / U0

/ext3/miniconda3/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


In [6]:
from xmovie import Movie
import matplotlib.ticker as tkr

def numfmt(x, pos):
    s = '{}'.format(int(x))
    return s
fmt = tkr.FuncFormatter(numfmt)

# This defines a custom plotting function that will plot each variable next to each other

Ld = 15e3
U0 = 0.01

fontsize = 16
def custom_plotfunc(ds, fig, i, *args, **kwargs):
    (ax1, ax2) = fig.subplots(ncols = 2, sharey=True)
    fig.set_figheight(7)
    fig.set_figwidth(18)
    
    t = ds.time.values
    ti = format(t[i] * U0 / Ld, '.2f')
    x = ds.x.values / Ld
    y = ds.y.values / Ld
    xx, yy = np.meshgrid(x, y)
    zeta1 = zeta.isel(time = i).isel(lev = 0)
    zeta2 = zeta.isel(time = i).isel(lev = 1)
    
    vmax = 60
    vmin = -vmax

    cmap = cm.cm.curl
    
    ax = ax1
    im = ax.pcolormesh(xx, yy, zeta1, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, upper layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel(r'$y / \lambda$', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

    vmax = 45
    vmin = -vmax
    
    ax = ax2
    im = ax.pcolormesh(xx, yy, zeta2, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, lower layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel('', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

/ext3/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
zeta = zeta.rename({'t': 'time'})
mov_custom = Movie(zeta, custom_plotfunc, input_check = False)
mov_custom.save(name + '_zeta_layer.mp4', framerate = 10, overwrite_existing = True)
zeta = zeta.rename({'time': 't'})

/ext3/miniconda3/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(


In [ ]:
name = 'kappa02_kt25_h12'
ds = expt_dict[name]
q = ds.q
psih = psih_func(a, q)
zeta = zeta_func(K2, psih, q) * Ld / U0

In [ ]:
from xmovie import Movie
import matplotlib.ticker as tkr

def numfmt(x, pos):
    s = '{}'.format(int(x))
    return s
fmt = tkr.FuncFormatter(numfmt)

# This defines a custom plotting function that will plot each variable next to each other

Ld = 15e3
U0 = 0.01

fontsize = 16
def custom_plotfunc(ds, fig, i, *args, **kwargs):
    (ax1, ax2) = fig.subplots(ncols = 2, sharey=True)
    fig.set_figheight(7)
    fig.set_figwidth(18)
    
    t = ds.time.values
    ti = format(t[i] * U0 / Ld, '.2f')
    x = ds.x.values / Ld
    y = ds.y.values / Ld
    xx, yy = np.meshgrid(x, y)
    zeta1 = zeta.isel(time = i).isel(lev = 0)
    zeta2 = zeta.isel(time = i).isel(lev = 1)
    
    vmax = 60
    vmin = -vmax

    cmap = cm.cm.curl
    
    ax = ax1
    im = ax.pcolormesh(xx, yy, zeta1, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, upper layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel(r'$y / \lambda$', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

    vmax = 45
    vmin = -vmax
    
    ax = ax2
    im = ax.pcolormesh(xx, yy, zeta2, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, lower layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel('', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

In [ ]:
zeta = zeta.rename({'t': 'time'})
mov_custom = Movie(zeta, custom_plotfunc, input_check = False)
mov_custom.save(name + '_zeta_layer.mp4', framerate = 10, overwrite_existing = True)
zeta = zeta.rename({'time': 't'})

In [5]:
Ld = 15e3
U0 = 1e-2

In [6]:
name = 'kappa02_GJ_h4'
ds = expt_dict[name]
q = ds.q
psih = psih_func(a, q)
zeta = zeta_func(K2, psih, q) * Ld / U0

/ext3/miniconda3/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


In [7]:
from xmovie import Movie
import matplotlib.ticker as tkr

def numfmt(x, pos):
    s = '{}'.format(int(x))
    return s
fmt = tkr.FuncFormatter(numfmt)

# This defines a custom plotting function that will plot each variable next to each other

Ld = 15e3
U0 = 0.01

fontsize = 16
def custom_plotfunc(ds, fig, i, *args, **kwargs):
    (ax1, ax2) = fig.subplots(ncols = 2, sharey=True)
    fig.set_figheight(7)
    fig.set_figwidth(18)
    
    t = ds.time.values
    ti = format(t[i] * U0 / Ld, '.2f')
    x = ds.x.values / Ld
    y = ds.y.values / Ld
    xx, yy = np.meshgrid(x, y)
    zeta1 = zeta.isel(time = i).isel(lev = 0)
    zeta2 = zeta.isel(time = i).isel(lev = 1)
    
    vmax = 300
    vmin = -vmax

    cmap = cm.cm.curl
    
    ax = ax1
    im = ax.pcolormesh(xx, yy, zeta1, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, upper layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel(r'$y / \lambda$', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

    vmax = 150
    vmin = -vmax
    
    ax = ax2
    im = ax.pcolormesh(xx, yy, zeta2, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, lower layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel('', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

/ext3/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
zeta = zeta.rename({'t': 'time'})
mov_custom = Movie(zeta, custom_plotfunc, input_check = False)
mov_custom.save(name + '_zeta_layer.mp4', framerate = 10, overwrite_existing = True)
zeta = zeta.rename({'time': 't'})

/ext3/miniconda3/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(


Movie created at kappa02_GJ_h4_zeta_layer.mp4


In [9]:
name = 'kappa02_GJ_h8'
ds = expt_dict[name]
q = ds.q
psih = psih_func(a, q)
zeta = zeta_func(K2, psih, q) * Ld / U0

/ext3/miniconda3/lib/python3.10/site-packages/xrft/xrft.py:573: FutureWarning: Default ifft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


In [10]:
from xmovie import Movie
import matplotlib.ticker as tkr

def numfmt(x, pos):
    s = '{}'.format(int(x))
    return s
fmt = tkr.FuncFormatter(numfmt)

# This defines a custom plotting function that will plot each variable next to each other

Ld = 15e3
U0 = 0.01

fontsize = 16
def custom_plotfunc(ds, fig, i, *args, **kwargs):
    (ax1, ax2) = fig.subplots(ncols = 2, sharey=True)
    fig.set_figheight(7)
    fig.set_figwidth(18)
    
    t = ds.time.values
    ti = format(t[i] * U0 / Ld, '.2f')
    x = ds.x.values / Ld
    y = ds.y.values / Ld
    xx, yy = np.meshgrid(x, y)
    zeta1 = zeta.isel(time = i).isel(lev = 0)
    zeta2 = zeta.isel(time = i).isel(lev = 1)
    
    vmax = 60
    vmin = -vmax

    cmap = cm.cm.curl
    
    ax = ax1
    im = ax.pcolormesh(xx, yy, zeta1, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, upper layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel(r'$y / \lambda$', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

    vmax = 45
    vmin = -vmax
    
    ax = ax2
    im = ax.pcolormesh(xx, yy, zeta2, vmin = -vmax, vmax = vmax, cmap = cmap)
    cbar = plt.colorbar(im, ax = ax, ticks = np.linspace(-vmax, vmax, 7, endpoint = True)) 
    # Tidy up the figure
    ax.set_title(f'relative vorticity, lower layer \n t = ' + ti + '$U / \lambda$', fontsize = fontsize) 
    ax.set_xlabel(r'$x / \lambda$', fontsize=fontsize)
    ax.set_ylabel('', fontsize=fontsize)
    ax.tick_params(axis='both', which='major', labelsize = fontsize)
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)

In [11]:
zeta = zeta.rename({'t': 'time'})
mov_custom = Movie(zeta, custom_plotfunc, input_check = False)
mov_custom.save(name + '_zeta_layer.mp4', framerate = 10, overwrite_existing = True)
zeta = zeta.rename({'time': 't'})

/ext3/miniconda3/lib/python3.10/site-packages/xmovie/core.py:348: UserWarning: The provided `plotfunc` does not provide the expected number of output arguments.            Expected a function `ax,pp =plotfunc(...)` but got %i output arguments. Inserting dummy values. This should not affect output. 
  warnings.warn(


Movie created at kappa02_GJ_h8_zeta_layer.mp4
